# 🌪️ Lorenz 80 (Simulations)

## 📝 Description

This code aims to simulate the Lorenz 80 model, published in the article [Attractor Sets and Quasi-Geostrophic Equilibrium](https://journals.ametsoc.org/view/journals/atsc/37/8/1520-0469_1980_037_1685_asaqge_2_0_co_2.xml). The model can be described using the equations below:

$ a_i \frac{dx_i}{d\tau} = a_i b_i x_i x_k - c (a_i - a_k) x_i y_k + c (a_i - a_j) y_i x_k - 2c^2 y_i y_k - \nu_0 a_i^2 x_i + a_i y_i - a_i z_i $

$ a_i \frac{dy_i}{d\tau} = - a_i b_k x_i y_k - a_i b_i y_i x_k + c (a_k - a_i) y_i y_k - a_i x_i - \nu_0 a_i^2 y_i $

$ \frac{dz_i}{d\tau} = - b_k x_i (z_k - h_k) - b_i (z_i - h_i) x_k + c\, y_i (z_k - h_k) - c (z_i - h_i) y_k + g_0 a_i x_i - \kappa_0 a_i z_i + F_i $

In [4]:
import numpy as np
from scipy.integrate import solve_ivp
import pandas as pd
from pathlib import Path


In [5]:
a = [1, 1, 3]
b = [
    0.5 * (a[0] - a[1] - a[2]),
    0.5 * (a[1] - a[2] - a[0]),
    0.5 * (a[2] - a[0] - a[1]),
]
c = np.sqrt(b[0]*b[1] + b[1]*b[2] + b[2]*b[0])

h = [-1, 0, 0]
f = [0.327, 0, 0]
g_0 = 8
kappa_0 = 1 / 48
nu_0 = kappa_0

In [6]:
def pe_model(t, u):
    x = u[0:3]
    y = u[3:6]
    z = u[6:9]
    dx = np.zeros(3)
    dy = np.zeros(3)
    dz = np.zeros(3)
    for i in range(3):
        j = (i + 1) % 3
        k = (i + 2) % 3
        dx[i] = (
            a[i] * b[i] * x[j] * x[k]
            - c * (a[i] - a[k]) * x[j] * y[k]
            + c * (a[i] - a[j]) * y[j] * x[k]
            - 2 * c**2 * y[j] * y[k]
            - nu_0 * (a[i] ** 2) * x[i]
            + a[i] * y[i]
            - a[i] * z[i]
        ) / a[i]
        dy[i] = (
            -a[k] * b[k] * x[j] * y[k]
            - a[j] * b[j] * y[j] * x[k]
            + c * (a[k] - a[j]) * y[j] * y[k]
            - a[i] * x[i]
            - nu_0 * (a[i] ** 2) * y[i]
        ) / a[i]
        dz[i] = (
            -b[k] * x[j] * (z[k] - h[k])
            - b[j] * (z[j] - h[j]) * x[k]
            + c * y[j] * (z[k] - h[k])
            - c * (z[j] - h[j]) * y[k]
            + g_0 * a[i] * x[i]
            - kappa_0 * a[i] * z[i]
            + f[i]
        )
    return np.concatenate([dx, dy, dz])

def simulate(x0, y0, z0, days): 
    initial_u = np.concatenate([x0, y0, z0])
    t_final = days * 8
    t_span = (0, t_final)
    sol = solve_ivp(pe_model, t_span, initial_u, method="RK45", atol=1e-8, rtol=1e-6)
    return sol.t / 8, sol.y[:3].T, sol.y[3:6].T, sol.y[6:].T


In [7]:
days = 400

# HARDLEY
y1 = f[0]/(a[0]*nu_0*(1+a[0]*g_0))
x1 = -nu_0*a[0]*y1
z1 = y1


x0 = [x1, 0, 0]
y0 = [y1, -(10 ** (-5)), 0]
z0 = [z1, 10 ** (-5), 0]

t, x, y, z = simulate(x0, y0, z0, days)

x = np.asarray(x)
y = np.asarray(y)
z = np.asarray(z)
t = np.squeeze(np.asarray(t))

df = pd.DataFrame(
    {
        "time": t,
        "x1": x[:, 0],
        "x2": x[:, 1],
        "x3": x[:, 2],
        "y1": y[:, 0],
        "y2": y[:, 1],
        "y3": y[:, 2],
        "z1": z[:, 0],
        "z2": z[:, 1],
        "z3": z[:, 2],
    }
)

df.to_csv("hardley_0327.csv", index=False)

